In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

from bs4 import BeautifulSoup
import requests
import csv
from random import randint
from time import sleep

In [2]:
with open('links.csv', newline='') as f:
    reader = csv.reader(f)
    your_list = list(reader)
links = []
for i in range(len(your_list)):
    links.append(your_list[i][0])
links = list(set(links))

In [3]:
def date_parse(script, day1, day2):
    parse = script[script.index(day1) + len(day1) + 2 : script.index(day2)-2]
    return parse.replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')

def date_add(date_str):
        count = 0
        for i in date_str:
            count += int(i)
        return count
    
def make_series(value, series_name):
    return pd.Series(value, name=series_name)

In [61]:
def scrap_url(url):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    soup_text = soup.prettify()
    
    try:
        description = soup.find('p', {"class":"spc-tiny medium-hide secondary-text light-header"}).text
    except:
        description = ''
    
    try:
        hour_str = soup.find('h3').text
        hours_tutor = int(hour_str[:hour_str.index(' hours')].replace(',',""))
    except: 
        hours_tutor = ''
    try:
        rate_str = soup.find('h4').div.contents[0]
        rate = int(rate_str[rate_str.index('$')+1:])
    except: 
        rate = ''
    
    try:   
        bio = ''
        for tag in soup.find_all('div', {"class":"full-content hide"}):
            bio = bio + tag.text
            bio = bio.replace('\n',"")
    except:
        bio = ''
    
    try:
        edu = []
        for tag in soup.find_all('section', {"class":"spc-med-s"}):
            item = str(tag.text).replace('\n', '').strip()
            item = " ".join(item.split())
            edu.append(item)
    except:
        edu = ''
        
    try:
        subjects_emph = set()
        for i in soup.find_all('a', {"class":"text-nowrap"}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_emph.add(item)
        subjects_emph.discard('Google Play')
        subjects_emph.discard('App Store')
        
    except:
        subjects_emph = ''
        
    try:
        subjects_all = set()
        for i in soup.find_all('span', {'class':'text-nowrap'}):
            item = str(i.text).replace('\n', '').strip()
            item = item.replace(',', '')
            item = " ".join(item.split())
            subjects_all.add(item)
    except:
        subjects_all = ''
        
    try:
        ratings_text = soup.find('a',{"class":"ratings-anchor"}).text.replace('(','').replace(')','').replace(',', '')
        ratings_split = ratings_text.split()
        num_ratings = int(ratings_split[0])
        num_unique_students = hours_tutor / num_ratings
    except:
        ratings_text = ''
        num_ratings = ''
        num_unique_students = ''
           
    try:
        cal = soup.find('div',{"class":"availabilities-container hide"}).script.text
        sun = date_parse(cal, 'sunday', 'monday')
        mon = date_parse(cal, 'monday', 'tuesday')
        tues = date_parse(cal, 'tuesday', 'wednesday')
        wed = date_parse(cal, 'wednesday', 'thursday')
        thurs = date_parse(cal, 'thursday', 'friday')
        fri = date_parse(cal, 'friday', 'saturday')
        sat = cal[cal.index('saturday') + 10 : -2].replace('[','').replace(']','').replace('false', "0").replace('true', "1").replace(',','')
        hours_sun = date_add(sun)
        hours_mon = date_add(mon)
        hours_tues = date_add(wed)
        hours_wed = date_add(wed)
        hours_thurs = date_add(thurs)
        hours_fri = date_add(fri)
        hours_sat = date_add(sat)
        hours_per_week_schedule = hours_sun + hours_mon + hours_tues + hours_wed + hours_thurs + hours_fri + hours_sat
        
    except:
        hours_sun = ''
        hours_mon = ''
        hours_tues = ''
        hours_wed = ''
        hours_thurs = ''
        hours_fri = ''
        hours_sat = ''
        hours_per_week_schedule = ''
        
    try:    
        bg_check = soup.find_all('p',{"class":'spc-zero'})[1].text
        bg_check_date = bg_check.replace('\n', '').split()[4]
        bg_check_date = datetime.strptime(bg_check_date, '%m/%d/%Y').date()
    
    except:
        bg_check_date = ''
    
    try:
        review_dates = []
        for i in range(len(soup_text)):
            if soup_text[i:i+11] == 'dateCreated':
                review_dates.append(soup_text[i:i+24])
    except:
        review_dates = ''
        
    descriptions = pd.Series(description, name='descriptions')
    hours_tutor = pd.Series(hours_tutor, name='hours_tutor')
    rate = pd.Series(rate, name='rate')
    bio = pd.Series(bio, name='bio')
    edu = pd.Series([edu], name='edu')
    subjects_emph = pd.Series([list(subjects_emph)], name='subjects_emph')
    subjects_all = pd.Series([list(subjects_all)], name='subjects_all')
    num_ratings = pd.Series(num_ratings, name='num_ratings')
    hours_sun = pd.Series(hours_sun, name='hours_sun')
    hours_mon = pd.Series(hours_mon, name='hours_mon')
    hours_tues = pd.Series(hours_tues, name='hours_tues')
    hours_wed = pd.Series(hours_wed, name='hours_wed')
    hours_thurs = pd.Series(hours_thurs, name='hours_thurs')
    hours_fri = pd.Series(hours_fri, name='hours_fri')
    hours_sat = pd.Series(hours_sat, name='hours_sat')
    hours_per_week_schedule = pd.Series(hours_per_week_schedule, name='hours_per_week_schedule')
    bg_check_date = pd.Series(bg_check_date, name='bg_check_date')
    review_dates = pd.Series([list(review_dates)], name='review_dates')
    
    scraped_data = pd.concat([descriptions, hours_tutor, rate, bio, num_ratings, subjects_emph, subjects_all, edu,
                       hours_sun, hours_mon, hours_tues, hours_wed, hours_thurs, hours_fri, hours_sat,
                       hours_per_week_schedule, bg_check_date, review_dates], axis=1)     
    return scraped_data

In [78]:
%%time
df = pd.DataFrame()
for i in range(10):
#for i in range(len(links)):
    scraped = scrap_url(links[i])
    df = df.append(scraped)
    sleep(randint(1,5))
#df.to_csv('test_tutor_data.csv')
df = df.reset_index(drop=True)

CPU times: user 2.12 s, sys: 36.5 ms, total: 2.15 s
Wall time: 40.7 s


In [72]:
days_since_last_review_series = pd.Series(name='days_since_last_review')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [79]:
df

,descriptions,hours_tutor,rate,bio,num_ratings,subjects_emph,subjects_all,edu,hours_sun,hours_mon,hours_tues,hours_wed,hours_thurs,hours_fri,hours_sat,hours_per_week_schedule,bg_check_date,review_dates
0,Columbia University Math and Computer Science ...,4,40,I am a senior at Columbia University studying ...,,[Computer Science],"[Precalculus, Calculus, Elementary Math, SAT M...",[Columbia University Math and Comp. Sci],7,2,2,2,3,7,5,28,,[]
1,Expert Tutor for High School or College Math,357,50,I've found one-on-one sessions as the most eff...,82,"[Precalculus, Statistics, Linear Algebra, Disc...","[Microsoft Excel, Trigonometry, Algebra 1, Pro...","[MIT Cambridge MA undergrad, Rutgers Universit...",9,9,9,9,9,9,9,63,2011-09-27,"[dateCreated"":""2016-11-07, dateCreated"":""2012-..."
2,Math and Almost Anything Tutor,471,40,I've been tutoring math since high school. In ...,109,"[Algebra 1, Algebra 2, Calculus]","[Literature, Spelling, English, Prealgebra, Ge...",[Arizona State University Mathematics],11,2,2,2,2,2,11,32,2015-04-06,"[dateCreated"":""2017-06-23, dateCreated"":""2017-..."
3,"Math, etc. Tutor",3,36,"Hello, I am a professional teacher with creden...",,"[Precalculus, Algebra 1, ACT Math, Algebra 2, ...","[Literature, Spanish, Spelling, English, Preal...","[Alumni of University of California, Los Angel...",4,8,12,12,12,12,12,72,,[]
4,Math Tutor Ranging from Elementary Math to Cal...,1,60,"I was born in Cerritos. At the age of 6, my da...",1,"[Precalculus, Algebra 1, Geometry, Algebra 2, ...","[Trigonometry, ACT Math, Probability, Elementa...",[University of California - Irvine Mathematics],7,0,9,9,7,9,15,56,,[]
5,"Engineering, Science and Language Wizard",39,25,Motivated and passionate about teaching and ma...,12,"[Spanish, Algebra 1, Italian, Mechanical Engin...","[Precalculus, Statistics, Algebra 2, Physics, ...",[Purdue University B.S. in Mechanical Engineer...,12,4,5,5,5,5,13,49,2019-11-10,"[dateCreated"":""2019-10-25, dateCreated"":""2019-..."
6,Patient Math/Language Tutor with Engineering B...,143,45,I am a process engineer in the biotech industr...,38,"[Precalculus, Study Skills, Trigonometry, Chem...","[Microsoft Excel, World History, Elementary Sc...",[Michigan State University Chemical Engineering],0,5,5,5,5,5,0,25,2015-04-18,"[dateCreated"":""2016-11-19, dateCreated"":""2016-..."
7,"Expert Chemistry, Math and Test Prep Help Avai...",29,60,I can help you! I have had great success tuto...,12,"[Algebra 1, Algebra 2, Prealgebra, Chemistry]","[PSAT, Trigonometry, SAT Math, Elementary Scie...","[Wesleyan University Chemistry, Columbia Unive...",10,13,14,14,11,8,8,78,2017-01-27,"[dateCreated"":""2017-03-12]"
8,CERTIFIED MATHEMATICS TEACHER,17,75,I have been teaching Mathematics for 14 years....,8,"[Precalculus, Trigonometry, ADHD, Algebra 1, G...",[],"[NYU UNIVERSITY Mathematics, Education, Africa...",16,7,7,7,7,7,16,67,2013-04-30,"[dateCreated"":""2014-01-15, dateCreated"":""2013-..."
9,"Tutor for Elem. and Middle Sc. Math, Algebra, ...",411,40,I strive to make learning fun and productive. ...,220,"[Algebra 1, Geometry, Prealgebra, GED, Physica...",[],[North Dak St University Math and Science Educ...,0,5,11,11,8,8,9,52,2013-07-02,"[dateCreated"":""2016-11-10, dateCreated"":""2016-..."


In [15]:
df.drop(df[df['num_ratings'] == ''].index, inplace = True)
today = date.today()
days_since_earliest_review_series = []
for i in df['review_dates']:
    list_dates = []
    for d in i:
        list_dates.append(datetime.strptime(d[14:], '%Y-%m-%d').date())
    print(min(list_dates))
    earliest_review = min(list_dates)
    #latest_review = max(c)
    days_since_earliest_review = (today - earliest_review).days
    days_since_earliest_review_series.append(days_since_earliest_review)
days_since_earliest_review_series = np.array(days_since_earliest_review_series)
df['hours_per_week_since_earliest_review'] = (df['hours_tutor'] / days_since_earliest_review_series) * 7 

2011-09-19
2016-02-29


ValueError: min() arg is an empty sequence

In [95]:
# drop review_dates that are null
length_dates = [len(i) for i in df['review_dates']]
for i in range(len(length_dates)):
    if length_dates[i] == 0:
        df.drop(i, inplace=True)

In [96]:
df

,descriptions,hours_tutor,rate,bio,num_ratings,subjects_emph,subjects_all,edu,hours_sun,hours_mon,hours_tues,hours_wed,hours_thurs,hours_fri,hours_sat,hours_per_week_schedule,bg_check_date,review_dates
1,Expert Tutor for High School or College Math,357,50,I've found one-on-one sessions as the most eff...,82,"[Precalculus, Statistics, Linear Algebra, Disc...","[Microsoft Excel, Trigonometry, Algebra 1, Pro...","[MIT Cambridge MA undergrad, Rutgers Universit...",9,9,9,9,9,9,9,63,2011-09-27,"[dateCreated"":""2016-11-07, dateCreated"":""2012-..."
2,Math and Almost Anything Tutor,471,40,I've been tutoring math since high school. In ...,109,"[Algebra 1, Algebra 2, Calculus]","[Literature, Spelling, English, Prealgebra, Ge...",[Arizona State University Mathematics],11,2,2,2,2,2,11,32,2015-04-06,"[dateCreated"":""2017-06-23, dateCreated"":""2017-..."
5,"Engineering, Science and Language Wizard",39,25,Motivated and passionate about teaching and ma...,12,"[Spanish, Algebra 1, Italian, Mechanical Engin...","[Precalculus, Statistics, Algebra 2, Physics, ...",[Purdue University B.S. in Mechanical Engineer...,12,4,5,5,5,5,13,49,2019-11-10,"[dateCreated"":""2019-10-25, dateCreated"":""2019-..."
6,Patient Math/Language Tutor with Engineering B...,143,45,I am a process engineer in the biotech industr...,38,"[Precalculus, Study Skills, Trigonometry, Chem...","[Microsoft Excel, World History, Elementary Sc...",[Michigan State University Chemical Engineering],0,5,5,5,5,5,0,25,2015-04-18,"[dateCreated"":""2016-11-19, dateCreated"":""2016-..."
7,"Expert Chemistry, Math and Test Prep Help Avai...",29,60,I can help you! I have had great success tuto...,12,"[Algebra 1, Algebra 2, Prealgebra, Chemistry]","[PSAT, Trigonometry, SAT Math, Elementary Scie...","[Wesleyan University Chemistry, Columbia Unive...",10,13,14,14,11,8,8,78,2017-01-27,"[dateCreated"":""2017-03-12]"
8,CERTIFIED MATHEMATICS TEACHER,17,75,I have been teaching Mathematics for 14 years....,8,"[Precalculus, Trigonometry, ADHD, Algebra 1, G...",[],"[NYU UNIVERSITY Mathematics, Education, Africa...",16,7,7,7,7,7,16,67,2013-04-30,"[dateCreated"":""2014-01-15, dateCreated"":""2013-..."
9,"Tutor for Elem. and Middle Sc. Math, Algebra, ...",411,40,I strive to make learning fun and productive. ...,220,"[Algebra 1, Geometry, Prealgebra, GED, Physica...",[],[North Dak St University Math and Science Educ...,0,5,11,11,8,8,9,52,2013-07-02,"[dateCreated"":""2016-11-10, dateCreated"":""2016-..."


In [82]:
df[df['review_dates'] == []].index()

ValueError: Lengths must match to compare

In [33]:
arr_len = [len(df['review_dates']) for i in df['review_dates']] 
#for i in df['review_dates']:
   # if i == []:
#df.drop(df[df['review_dates'] == [].index, inplace = True)

In [34]:
arr_len

[8, 8, 8, 8, 8, 8, 8, 8]

In [8]:
url = 'https://www.wyzant.com/match/tutor/86182268?search_id=10258f52-7036-483d-87d3-7852f0c963c2'



In [25]:
c = []
for i in scrap_url(url).review_dates[0]:
    c.append(datetime.strptime(i[14:], '%Y-%m-%d').date())
min(c)
df_url['earliest_review'] = min(c)
df_url['latest_review'] = max(c)

In [50]:
today = date.today()
df_url['days_since_earliest_review'] = (today - df_url['earliest_review'][0]).days
df_url['days_since_latest_review'] = (today - df_url['latest_review'][0]).days
df_url['hours_per_week_since_earliest_review'] = (df_url['hours_tutor'] / df_url['days_since_earliest_review']) * 7

In [69]:
df_url['description_length'] = len(df_url['descriptions'][0].split())
df_url['bio_length'] = len(df_url['bio'][0].split())
df_url['number_subjects'] = len(df_url['subjects_emph'][0]) + len(df_url['subjects_all'][0])
df_url['number_degrees'] = len(df_url['edu'][0])

In [70]:
df_url

,descriptions,hours_tutor,rate,bio,num_ratings,subjects_emph,subjects_all,edu,hours_sun,hours_mon,...,review_dates,earliest_review,latest_review,days_since_earliest_review,days_since_latest_review,hours_per_week_since_earliest_review,description_length,bio_length,number_subjects,number_degrees
0,"Expert mathematician, Functional programmer, T...",239,79,*** Me ***Mathematician turned software (block...,77,"[Logic, Discrete Math, Computer Programming, C...","[Algebra 2, Prealgebra, Algebra 1, Precalculus...","[FAU Honors College Mathematics and physics, K...",5,10,...,"[dateCreated"":""2020-03-28, dateCreated"":""2020-...",2020-03-28,2020-05-28,67,6,24.970149,7,287,13,3


In [ ]:
cols_to_drop = ['hours_tutor', 'num_ratings']
df.drop(columns=cols_to_drop, inplace=True)